[View in Colaboratory](https://colab.research.google.com/github/JozeeLin/google-tensorflow-exercise/blob/master/%E7%BB%8F%E5%85%B8%E5%8D%B7%E7%A7%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C_VGGNet.ipynb)

## VGGNet
VGGNet是牛津大学计算机视觉组和google deepmind公司的研究员一起研发的深度卷积神经网络。VGGNet探索了卷积神经网络的深度与其性能之间的关系，

通过反复堆叠3x3的小型卷积核和2x2的最大池化层，VGGNet成功的构筑了16~9层深的卷积神经网络。

VGGNet论文中全部使用了3x3的卷积核和2x2的池化核，通过不断加深网络结构来提升性能。

虽然从A到E每一级网络逐渐变深，但是网络的参数量并没有增长很多，这是因为参数量主要都消耗在最后3个全连接层。

前面的卷积部分虽然很深，但是消耗的参数量不大，不过训练比较耗时的部分依然是卷积，因其计算量比较大。C很有意思，相比B多了几个1x1的卷积层，1x1卷积的意义主要在于线性变换，

而输入通道数和输出通道数不变，没有发生降维。

VGGNet拥有5段卷积，每一段内有2~3个卷积层，同时每段尾部会连接一个最大池化层用来缩小图片尺寸。

VGGNet在训练时有一个小技巧，先训练级别A的简单网络，再复用A网络的权重来初始化后面的几个复杂模型，这样训练收敛的速度更快。

在预测时，VGGNet采用Multi-Scale的方法，将图像scale到一个尺寸Q，并将图像输入卷积网络计算。

然后，在最后一个卷积层使用滑窗的方式进行分类预测，将不同窗口的分类结果平均，再将不同尺寸Q的结果平均得到最后结果，这样可提高图片数据的利用率并提升预测准确率

同时，在训练中，VGGNet使用Multi-Scale的方法做数据增强，将原始图像缩放到不同尺寸S，然后在随机裁切224x224的图片，这样能增加很多数据量，对于方式模型过拟合有很不错的效果。

VGGNet作者在对比各级网络时总结出以下几个观点：
- LRN层作用不大
- 越深的网络效果越好
- 1x1的卷积也是很有效的，但是没有3x3的卷积好，大一些的卷积核可以学习更大的空间特征

In [0]:
from datetime import datetime
import math
import time
import tensorflow as tf

In [0]:
def conv_op(input_op, name, kh, kw, n_out, dh, dw, p):
  n_in = input_op.get_shape()[-1].value
  
  with tf.name_scope(name) as scope:
    kernel = tf.get_variable(scope+'w', shape=[kh,kw,n_in,n_out],
                            dtype=tf.float32,
                            initializer=tf.contrib.layers.xavier_initializer_conv2d())
    
    conv = tf.nn.conv2d(input_op, kernel, (1,dh,dw,1), padding='SAME')
    bias_init_val = tf.constant(0.0, shape=[n_out], dtype=tf.float32)
    biases = tf.Variable(bias_init_val, trainable=True, name='b')
    z = tf.nn.bias_add(conv, biases)
    activation = tf.nn.relu(z, name=scope)
    p += [kernel, biases]
    return activation

In [0]:
def fc_op(input_op, name, n_out, p):
  n_in = input_op.get_shape()[-1].value
  
  with tf.name_scope(name) as scope:
    kernel = tf.get_variable(scope+'w',shape=[n_in, n_out], dtype=tf.float32,
                            initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.Variable(tf.constant(0.1, shape=[n_out],dtype=tf.float32), name='b')
    activation = tf.nn.relu_layer(input_op, kernel, biases, name=scope)
    p += [kernel, biases]
    return activation

In [0]:
#最大池化层的创建函数mpool_op
def mpool_op(input_op, name, kh, kw, dh, dw):
  return tf.nn.max_pool(input_op, ksize=[1,kh,kw,1],
                       strides=[1,dh,dw,1],
                       padding='SAME',
                       name=name)

In [0]:
#接下来开始创建VGGNet-16的网络结构。主要分为6部分，前5段为卷积网络，最后一段是全连接网络。
def inference_op(input_op, keep_prob):
  #参数列表
  p = []
  #第一段卷积网络，由两个卷积层和一个最大池化层构成
  conv1_1 = conv_op(input_op, name='conv1_1', kh=3, kw=3, n_out=64,dh=1,dw=1,p=p)
  conv1_2 = conv_op(conv1_1, name='conv1_2',kh=3, kw=3, n_out=64,dh=1,dw=1,p=p)
  pool1 = mpool_op(conv1_2, name='pool1', kh=2,kw=2,dw=2,dh=2)
  
  #第二段卷积网络
  conv2_1 = conv_op(pool1, name='conv2_1', kh=3, kw=3, n_out=128, dh=1, dw=1, p=p)
  conv2_2 = conv_op(conv2_1, name='conv2_2', kh=3, kw=3, n_out=128, dh=1, dw=1, p=p)
  pool2 = mpool_op(conv2_2, name='pool2', kh=2, kw=2, dh=2, dw=2)
  
  #第三段卷积网络,三个卷积层和一个最大池化层
  conv3_1 = conv_op(pool2, name='conv3_1', kh=3, kw=3, n_out=256, dh=1, dw=1, p=p)
  conv3_2 = conv_op(conv3_1, name='conv3_2', kh=3, kw=3, n_out=256, dh=1, dw=1, p=p)
  conv3_3 = conv_op(conv3_2, name='conv3_3', kh=3, kw=3, n_out=256, dh=1, dw=1, p=p)
  pool3 = mpool_op(conv3_3, name='pool3', kh=2,kw=2,dh=2,dw=2)
  
  #第四段卷积网络，三个卷积层和一个最大池化层
  conv4_1 = conv_op(pool3, name='conv4_1', kh=3, kw=3, n_out=512, dh=1,dw=1,p=p)
  conv4_2 = conv_op(conv4_1, name='conv4_2', kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
  conv4_3 = conv_op(conv4_2, name='conv4_3', kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
  pool4 = mpool_op(conv4_3, name='pool4', kh=2, kw=2, dh=2, dw=2)
  
  #最后一段卷积网络，三个卷积层和一个最大池化层
  conv5_1 = conv_op(pool4, name='conv5_1', kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
  conv5_2 = conv_op(conv5_1, name='conv5_2', kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
  conv5_3 = conv_op(conv5_2, name='conv5_3', kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
  pool5 = mpool_op(conv5_3, name='pool5', kh=2, kw=2, dw=2, dh=2)
  
  #将第五段卷积网络的输出结果进行扁平化，使用tf.reshape函数将每个样本化为长度为7x7x512=25088的一维向量
  shp = pool5.get_shape()
  flattened_shape = shp[1].value * shp[2].value * shp[3].value
  resh1 = tf.reshape(pool5, [-1, flattened_shape], name='resh1')
  
  #最后一层，全连接层
  fc6 = fc_op(resh1, name='fc6', n_out=4096, p=p)
  fc6_drop = tf.nn.dropout(fc6, keep_prob, name='fc6_drop')
  
  fc7 = fc_op(fc6_drop, name='fc7', n_out=4096, p=p)
  fc7_drop = tf.nn.dropout(fc7, keep_prob, name='fc7_drop')
  
  fc8 = fc_op(fc7_drop, name='fc8', n_out=1000, p=p)
  softmax = tf.nn.softmax(fc8)
  predictions = tf.argmax(softmax, 1)
  
  return predictions, softmax, fc8, p

In [0]:
def time_tensorflow_run(session, target, feed, info_string):
  num_steps_burn_in = 10
  total_duration = 0.0
  total_duration_squared = 0.0
  
  for i in range(num_batches + num_steps_burn_in):
    start_time = time.time()
    _ = session.run(target, feed_dict=feed)
    duration = time.time() - start_time
    if i>= num_steps_burn_in:
      if not i % 10:
        print('%s: step %d duration=%.3f' % (datetime.now(), i-num_steps_burn_in, duration))
        
      total_duration += duration
      total_duration_squared += duration * duration
      
  mn = total_duration / num_batches
  vr = total_duration_squared / num_batches - mn * mn
  sd = math.sqrt(vr)
  
  print('%s: %s across %d steps, %.3f +/- %.3f sec / batch' % (datetime.now(), info_string, num_batches, mn, sd))

In [0]:
def run_benchmark():
  with tf.Graph().as_default():
    image_size = 224
    images = tf.Variable(tf.random_normal([batch_size,
                                          image_size,
                                          image_size, 3],
                                         dtype=tf.float32,
                                         stddev=1e-1))
    
    keep_prob = tf.placeholder(tf.float32)
    predictions, softmax, fc8, p = inference_op(images, keep_prob)
    
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    
    time_tensorflow_run(sess, predictions, {keep_prob:1.0}, "Forward")
    objective = tf.nn.l2_loss(fc8)
    grad = tf.gradients(objective, p)
    time_tensorflow_run(sess, grad, {keep_prob:0.5}, "Forward-backward")

In [8]:
batch_size = 32
num_batches = 100
run_benchmark()

2018-05-03 13:49:57.493480: step 0 duration=0.279
2018-05-03 13:50:00.297833: step 10 duration=0.283
2018-05-03 13:50:03.102962: step 20 duration=0.282
2018-05-03 13:50:05.910167: step 30 duration=0.278
2018-05-03 13:50:08.715329: step 40 duration=0.281
2018-05-03 13:50:11.527550: step 50 duration=0.281
2018-05-03 13:50:14.336877: step 60 duration=0.280
2018-05-03 13:50:17.154626: step 70 duration=0.283
2018-05-03 13:50:19.962067: step 80 duration=0.279
2018-05-03 13:50:22.773752: step 90 duration=0.282
2018-05-03 13:50:25.301993: Forward across 100 steps, 0.281 +/- 0.001 sec / batch
2018-05-03 13:50:41.194457: step 0 duration=0.955
2018-05-03 13:50:50.717254: step 10 duration=0.954
2018-05-03 13:51:00.266689: step 20 duration=0.956
2018-05-03 13:51:09.799978: step 30 duration=0.956
2018-05-03 13:51:19.345971: step 40 duration=0.959
2018-05-03 13:51:28.876981: step 50 duration=0.962
2018-05-03 13:51:38.408970: step 60 duration=0.955
2018-05-03 13:51:47.958479: step 70 duration=0.960
20

VGGNet的模型参数虽然比AlexNet多，但反而只需要较少的迭代次数就可以收敛，主要原因是更深的网络和更小的卷积核带来的额隐式的正则化效果。